In [ ]:
!pip install pystac_client
!pip install planetary_computer
!pip install rasterio
!pip install xarray-spatial

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict
from pystac_client import Client
import planetary_computer
import xarray
import time

### Import Base Data Files

In [ ]:
ground_measures_metadata = pd.read_csv('/content/drive/MyDrive/snocast/eval/ground_measures_metadata.csv')
submission_format = pd.read_csv('/content/drive/MyDrive/snocast/eval/submission_format.csv')

In [ ]:
# get latitude longitude for train and test grids
f = open('/content/drive/MyDrive/snocast/eval/grid_cells.geojson')
grid_cells = json.load(f)
print('length grid_cells features: ', len(grid_cells['features']))

In [ ]:
ids = []
lats = []
lons = []
regions = []
bboxes = []

for grid_cell in grid_cells['features']:
    cell_id = grid_cell['properties']['cell_id']
    region = grid_cell['properties']['region']
    coordinates = grid_cell['geometry']['coordinates'][0]
    lon, lat = np.mean(coordinates, axis=0)
    northeast_corner = np.max(coordinates, axis=0)
    southwest_corner = np.min(coordinates, axis=0)
    # bbox = [min_lon, min_lat, max_lon, max_lat]
    bbox = np.concatenate([southwest_corner,northeast_corner])
    ids.append(cell_id)
    lats.append(lat)
    lons.append(lon)
    regions.append(region)
    bboxes.append(bbox)

grid_cells_pd = pd.DataFrame({'cell_id': ids, 
                             'latitude': lats, 
                             'longitude': lons,
                             'region': regions, 
                             'bbox': bboxes})

## Get Data for Copernicus Digital Elevation Model (DEM)

In [ ]:
client = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    ignore_conformance=True,
)

In [ ]:
all_max_lat = grid_cells_pd.latitude.max()
all_min_lat = grid_cells_pd.latitude.min()
all_max_lon = grid_cells_pd.longitude.max()
all_min_lon = grid_cells_pd.longitude.min()
all_bbox = [all_min_lon, all_min_lat, all_max_lon, all_max_lat]
print(all_min_lon, all_min_lat, all_max_lon, all_max_lat)

In [ ]:
search = client.search(
      collections=["cop-dem-glo-30"],
      bbox=all_bbox,
  )

items = list(search.get_items())
if len(items) > 1:
  print(f"Returned {len(items)} items")

In [ ]:
# Ran in 30 min. for 295 items
processed_items = []
for i in range(len(items)):
  signed_asset = planetary_computer.sign(items[i].assets["data"])
  data = (
      xarray.open_rasterio(signed_asset.href)
      .squeeze()
      .drop("band")
      .coarsen({"y": 5, "x": 5})
      .mean()
  )
  processed_items.append(data)

In [ ]:
mean_elevations = []
var_elevations = []

for idx, row in grid_cells_pd.iterrows():
  # if idx < 2263:
  #   continue
  if idx % 100 == 0:
    print(idx)
  min_lon, min_lat, max_lon, max_lat = row['bbox']

  sample_elevations = np.array([])
  for data in processed_items:
    lat_values = (data.y.values < max_lat) & (data.y.values > min_lat)
    lon_values = (data.x.values < max_lon) & (data.x.values > min_lon)
    mask = lon_values[np.newaxis, :] & lat_values[:, np.newaxis]
    sample_elevations = np.concatenate([sample_elevations, data.values[mask]])
  mean_elevation_m = sample_elevations.mean()
  var_elevation_m = sample_elevations.var()
  mean_elevations.append(mean_elevation_m)
  var_elevations.append(var_elevation_m)

In [ ]:
print(idx)
print(len(var_elevations))
print(len(mean_elevations))

In [ ]:
grid_cells_pd['elevation_m'] = mean_elevations
grid_cells_pd['elevation_var_m'] = var_elevations

In [ ]:
grid_cells_pd = grid_cells_pd[['cell_id', 'latitude', 'longitude', 'region', 'elevation_m','elevation_var_m']]

In [ ]:
grid_cells_pd.sample(3)

In [ ]:
grid_cells_pd.to_parquet('/content/drive/MyDrive/snocast/eval/grid_cells_elev.parquet')